In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
from utils import *

In [ ]:
# Config plotting:
mpl.rcParams['ps.useafm'] = True                                                     
#rc('font',**{'family':'sans-serif','sans-serif':['FreeSans']}) 
mpl.rcParams['pdf.fonttype'] = 3

### Read schedule

In [ ]:
# Initialize
schedule = toast.schedule.GroundSchedule()

In [ ]:
schedule.site_name

The schedule is created with ${\tt make\_schedule.sh}$ (see [Sec. 3](Sec3_Make_and_Analyze_schedule.ipynb)) or imported from [pwg-tds/pipe-s0002/v6/schedules/schedule_sat.txt](https://github.com/simonsobs/pwg-scripts/blob/24a8c8202e2f80fb9b5097ee0e2dcfe5c1c07114/pwg-tds/pipe-s0002/v6/schedules/schedule_sat.txt). 

In [ ]:
# Read
#schedule.read('schedules/schedule_sat_1day.txt') # generated with make_schedule.sh
#schedule.read('schedule_sat_pipe-s0002-v6.txt') # Full SAT schedule for 1 yr (74 observations)
#schedule.read('schedules/split_schedule/schedule_029.txt') # Split schedule for 1 observation
schedule.read('schedules/schedule_sat_1day.txt')

In [ ]:
schedule.telescope_name

### Create focalplane

Here you can select the wafer slots, tube slots, and whether to thin the focal plane.

In [ ]:
sample_rate = 40. #10. #Hz

In [ ]:
focalplane = sotoast.SOFocalplane(
        hwfile=None,
        telescope='SAT1',
        sample_rate=sample_rate * u.Hz,
        bands='SAT_f090',
        wafer_slots='w25', 
        tube_slots=None,
        thinfp=8,
        comm=None,
    )

### Create telescope from focalplane and schedule

In [ ]:
telescope = toast.Telescope(name="SAT1", 
                            focalplane=focalplane, 
                            site=toast.GroundSite("Atacama", schedule.site_lat,
                                            schedule.site_lon, schedule.site_alt))

### Create data object

In [ ]:
data = toast.Data()

In [ ]:
# This should be empty 
data 

### Simulate a generic ground-based telescope scanning

This simulates ground-based pointing in constant elevation scans for a telescope located at a particular site and using a pre-created schedule. Here we include HWP.

In [ ]:
_, sim_gnd = apply_scanning(data, telescope, schedule)

The data object will now contain (empty) observation instances, which otherwise can be uploaded from: 
- the context.yaml file (output of toast3 simulation, v4)
- the h5 file (output of toast3 simulation, v6)

In [ ]:
# Inspect the data
#data.info

In [ ]:
# Extract single instances from data.obs list, e.g.
#data.obs[0].telescope
for no, do in enumerate(data.obs):
    print(data.obs[no].telescope)

In [ ]:
# Signal should be empty, check first detector
data.obs[0].detdata['signal'][0]

In [ ]:
# Explore the data manager instance
data.obs[0].shared

In [ ]:
# Plot
fig,ax = plt.subplots(1,1,figsize=(6,1))
ax.plot(data.obs[0].shared['times'][:100], data.obs[0].shared['hwp_angle'][:100])
ax.set_xlabel('sec')
ax.set_ylabel('HWP angle [rad]')

fig,ax = plt.subplots(1,1,figsize=(6,1))
ax.plot(data.obs[0].shared['times'], np.array(data.obs[0].shared['azimuth'])*180./np.pi)
ax.set_xlabel('sec')
ax.set_ylabel('Azimuth [deg]')

fig,ax = plt.subplots(1,1,figsize=(6,1))
ax.plot(data.obs[0].shared['times'], data.obs[0].shared['elevation'])
ax.set_xlabel('sec')
ax.set_ylabel('Elevation')

### Pointing

In [ ]:
data, det_pointing_radec = apply_det_pointing_radec(data, sim_gnd)
data, det_pointing_azel = apply_det_pointing_azel(data, sim_gnd)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,2))
ax.plot(data.obs[0].shared['times'], np.array(data.obs[0].shared['azimuth'])*180./np.pi)
ax.set_ylabel('azimuth [deg]')

### Pixel pointing healpix

In [ ]:
nside_out = 256 #2048 #256 #64 512

In [ ]:
data, pixels_radec = apply_pixels_radec(data, det_pointing_radec, nside_out)

### Stokes Weights Radec

In [ ]:
data, weights_radec = apply_weights_radec(data, det_pointing_radec)

### Add signal

Here we use as input the HEALPix format map that was generated in [Sec. 2](Sec2_Sky_maps.ipynb) and scan it to a detector timestream.

In [ ]:
# Input map: signal only
file = f'input_maps/cmb_SAT_f093_ns{nside_out}.fits' #mine
#file = f'input_maps/reijo/cmb_SAT_f090.fits' #reijo's
IQUmap = hp.read_map(file, field=[0,1,2])

In [ ]:
npol = 3
for p in np.arange(npol):
    hp.mollview(IQUmap[p])

In [ ]:
data, scan_map = apply_scan_map(data, file, pixels_radec, weights_radec)

### Add noise

In [ ]:
_, noise_model = apply_noise_model(data)

In [ ]:
data, sim_noise = apply_sim_noise(data)

### Save HDF5

In [ ]:
save_hdf5 = toast.ops.SaveHDF5(name="save_hdf5")
#hdf5_path = os.path.join('outputs', "hdf5")
hdf5_path = 'outputs'
#hdf5_path = 'outputs_reijo'
if not os.path.isdir(hdf5_path):
    os.makedirs(hdf5_path)
save_hdf5.volume = hdf5_path
save_hdf5.apply(data)

In [ ]:
# To load
import toast
import toast.io
# Manage MPI
# Take the global number of processes available (MPI.COMM_WORLD) 
# and divide them into groups. Each process group is assigned one or more observations. 
toast_comm = toast.Comm() #default: world=None, groupsize=0

In [ ]:
ob = toast.io.load_hdf5(f'{hdf5_path}/obs_south-0-16_2884568106.h5', toast_comm)

In [ ]:
# Inspect
ob

### Make it into context file

This can easily be read in AxisManager form, for pre-processing purposes.

In [ ]:
import write_context

In [ ]:
export_dirs = [f"{hdf5_path}/"]  # Directory to search for HDF data files
context_dir = f"{export_dirs[0]}"  #context_south-0-29"  # Change this to desired output context directory
if not os.path.isfile(f'{context_dir}context.yaml'):
    write_context.create_context(context_dir, export_dirs) #TODO: currently exportdir and contextdir

In [ ]:
#TODO: currently exportdir and contextdir need to match for Sec.5 to read h5 file

#### Continue to the next section

Go to [Section 5 - Pre-process the data and make maps](Sec5_Preprocess_TOD_Make_maps.ipynb).